# Face and Emotion Detector

### Making Image Data Generator

In [2]:
from keras.preprocessing.image import ImageDataGenerator

img_w, img_h = 48, 48
batch_size = 16

train_data_dir = './fer2013/train'
test_data_dir  = './fer2013/validation'

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        horizontal_flip = True,
        rotation_range = 30,
        width_shift_range = 0.3,
        height_shift_range = 0.3,
        shear_range = 0.2,
        zoom_range = 0.2,
        fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
        rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_w, img_h),
        batch_size = batch_size,
        color_mode = 'grayscale',
        class_mode = 'categorical',
        shuffle = True)

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size = (img_w, img_h),
        batch_size = batch_size,
        color_mode = 'grayscale',
        class_mode = 'categorical',
        shuffle = True)

Found 28273 images belonging to 6 classes.
Found 3534 images belonging to 6 classes.


### Making the model, with required imports and declarations

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D

num_classes = 6

model = Sequential()

# 1st 2x(Conv-eLu)-Pooling layer with 0.2 Dropout (32 filters)
model.add(Conv2D(32, (3, 3), input_shape = (img_w, img_h, 1),
                 padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), input_shape = (img_w, img_h, 1),
                 padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 2nd 2x(Conv-eLu)-Pooling layer with 0.2 Dropout (64 filters)
model.add(Conv2D(64, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 3rd 2x(Conv-eLu)-Pooling layer with 0.2 Dropout (128 filters)
model.add(Conv2D(128, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 4th 2x(Conv-eLu)-Pooling layer with 0.2 Dropout (256 filters)
model.add(Conv2D(256, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# 5th 2x(Conv-eLu)-Pooling layer with 0.2 Dropout (512 filters)
model.add(Conv2D(512, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding = 'same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))

# Dense layers, with eLu activation

model.add(Flatten())
model.add(Dense(64, kernel_initializer = 'he_normal', activation = 'elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, kernel_initializer = 'he_normal', activation = 'elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Final Output Dense Layer

model.add(Dense(num_classes, kernel_initializer = 'he_normal', activation = 'softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
__________

In [5]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

epochs = 10

checkpoint = ModelCheckpoint('emotion_detection.h5',
                             monitor = 'val_loss',
                             mode = 'min',
                             save_best_only = True)

early_stopping = EarlyStopping(monitor = 'val_loss',
                               min_delta = 0,
                               patience = 3,
                               restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              min_delta = 0.0001)

callbacks = [checkpoint, early_stopping, reduce_lr]

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.0007),
              metrics = ['accuracy'])

num_training_samples = 28273
num_test_samples     = 3534

model.fit_generator(
        train_generator,
        steps_per_epoch = num_training_samples // batch_size,
        epochs = epochs,
        callbacks = callbacks,
        validation_steps = num_test_samples // batch_size,
        validation_data = test_generator)

Epoch 1/10
1767/1767 [==============================] - 4137s 2s/step - loss: 2.0246 - acc: 0.2018 - val_loss: 1.7635 - val_acc: 0.2514
Epoch 2/10
1767/1767 [==============================] - 4059s 2s/step - loss: 1.7489 - acc: 0.2464 - val_loss: 1.7424 - val_acc: 0.2609
Epoch 3/10
1767/1767 [==============================] - 4080s 2s/step - loss: 1.7004 - acc: 0.2773 - val_loss: 1.6882 - val_acc: 0.3002
Epoch 4/10
1767/1767 [==============================] - 4122s 2s/step - loss: 1.6161 - acc: 0.3379 - val_loss: 1.5010 - val_acc: 0.4093
Epoch 5/10
1767/1767 [==============================] - 4056s 2s/step - loss: 1.5066 - acc: 0.3968 - val_loss: 1.4258 - val_acc: 0.4377
Epoch 6/10
1767/1767 [==============================] - 4060s 2s/step - loss: 1.4373 - acc: 0.4332 - val_loss: 1.4154 - val_acc: 0.4534
Epoch 7/10
1767/1767 [==============================] - 4062s 2s/step - loss: 1.3875 - acc: 0.4536 - val_loss: 1.4597 - val_acc: 0.4625
Epoch 8/10
1767/1767 [==========================

KeyboardInterrupt: 

In [4]:
from keras.models import load_model

classifier = load_model('emotion_detection.h5')

validation_generator = test_datagen.flow_from_directory(
        test_data_dir,
        color_mode = 'grayscale',
        target_size=(img_w, img_h),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

Found 3534 images belonging to 6 classes.


In [5]:
import cv2
import numpy as np
from time import sleep
from keras.preprocessing.image import img_to_array

face_classifier = cv2.CascadeClassifier('./Haarcascades/haarcascade_frontalface_default.xml')

def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    if np.sum([face]) != 0.0:
        roi = face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        preds = classifier.predict(roi)[0]
        label = class_labels[preds.argmax()]  
        label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
        cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
    else:
        cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        
    cv2.imshow('All', image)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()